In [1]:
from csv import reader
from random import seed, randrange


seed(42)


def load_csv(file_path):
  dataset = []

  with open(file_path, "r") as file:
    csv_reader = reader(file)
    for row in csv_reader:
      if row:
        dataset.append(row)
  
  return dataset


def str_col_to_float(dataset, col):
  for row in dataset:
    row[col] = float(row[col].strip())


def mean(vals):
  return sum(vals) / float(len(vals))


def variance(vals):
  m = mean(vals)
  return sum(lambda v: pow(v - m, 2), vals)